In [24]:
import sys
sys.path.append('/home/jovyan/gitlab/gopay-genie-model_pipeline-staging')
print(sys.path)

import os
os.chdir('/home/jovyan/gitlab/gopay-genie-model_pipeline-staging')
print(os.getcwd())

['/opt/conda/lib/python39.zip', '/opt/conda/lib/python3.9', '/opt/conda/lib/python3.9/lib-dynload', '', '/home/jovyan/venv/gopay-genie-rio/lib/python3.9/site-packages', '/home/jovyan/gitlab/gopay-genie-model_pipeline-staging', '/home/jovyan/gitlab/gopay-genie-model_pipeline-staging', '/home/jovyan/gitlab/gopay-genie-model_pipeline-staging', '/home/jovyan/gitlab/gopay-genie-model_pipeline-staging']
/home/jovyan/gitlab/gopay-genie-model_pipeline-staging


In [25]:
import logging
import os
import pandas as pd
import numpy as np
import pandas_gbq as pdgbq
import subprocess
import time
from recommenders.models.sar import SAR
import re
from sklearn.preprocessing import StandardScaler
from recommenders.utils.timer import Timer
from gopay_genie.src.utils.data_utils import save_to_bigquery 
from gopay_genie.src.transformers.one_personalisation.config import RUN_DATE, END_DATE
from gopay_genie.src.transformers.config import BQ_PROJECT_ID, BQ_BASE_DATASET
from gopay_genie.src.transformers.one_personalisation.config import USER_BASE_TABLE_NAME, \
  TRAIN_SET_TABLE_NAME_30D, SERVE_DISK_DATA_DIR, sar_model_params, selected_items_sar,FINAL_UM_1PD_3PD_TABLE_NAME,LOG_FILE_NAME,TEMP_SAR_1PD_3PD_RESULT_TABLE_NAME, RESULT_TABLE_NAME, SERVE_GCS_DATA_DIR, TRAIN_SET_TABLE_NAME_RT, USER_BASE_DATE_rt, RTUM_3PD_sql_file, train_set_sql_file, real_time_um_sql_file, \
TRAIN_SET_TABLE_NAME_CLICK, click_signal_um_sql_file, NEW_EXPAND_USER_BASE_TABLE_NAME,train_set_v2_sql_file, current_ts, UM_3PD_sql_file, TABLE_NAME_3PD_90D, TABLE_NAME_3PD_30D, TABLE_NAME_3PD_RT
from gopay_genie.src.utils.data_collection import query_to_bq
from gopay_genie.src.utils.data_utils import read_sql_file
from gopay_genie.src.transformers.one_personalisation.dev_lightfm_complex_hyperparam.predict import \
    create_predict_df
from gopay_genie.src.transformers.one_personalisation.dev_lightfm_complex_hyperparam.evaluation import \
    metric_calc_spark

In [49]:
import logging
import pandas as pd
import numpy as np
import pandas_gbq as pdgbq
import subprocess
import time
from recommenders.models.sar import SAR
import re
from sklearn.preprocessing import StandardScaler
from recommenders.utils.timer import Timer
from gopay_genie.src.utils.data_utils import save_to_bigquery 
from gopay_genie.src.transformers.one_personalisation.config import RUN_DATE, END_DATE
from gopay_genie.src.transformers.config import BQ_PROJECT_ID, BQ_BASE_DATASET
from gopay_genie.src.transformers.one_personalisation.config import selected_items_sar, FINAL_NEW_USER_EXPAND_UM_TABLE_NAME, NON_MT_TEMP_SAR_1P_NEW_USER_RESULT_TABLE_NAME
from gopay_genie.src.utils.data_collection import query_to_bq
from gopay_genie.src.utils.data_utils import read_sql_file
from gopay_genie.src.utils.data_utils import save_to_bigquery_partition
from gopay_genie.src.transformers.one_personalisation.config import sar_model_params, sar_trx_count_thres, LOG_FILE_NAME
from gopay_genie.src.transformers.one_personalisation.dev_lightfm_complex_hyperparam.predict import create_predict_df
from gopay_genie.src.transformers.one_personalisation.dev_lightfm_complex_hyperparam.evaluation import metric_calc_spark
from gopay_genie.src.transformers.one_personalisation.utils import create_spark_session

In [35]:
user_base_sql = f"""
        select * from `gopay-ds-staging.playground_us.combined_user_list`
        """

In [26]:
RUN_DATE

'2024-11-21'

In [27]:
END_DATE

'2024-11-20'

In [81]:
um_query = read_sql_file(sql_path=train_set_sql_file).format(trx_end_date=END_DATE,
                                                          time_period = 90,
                                                          user_base_sql=user_base_sql)
gtum_query = read_sql_file(sql_path=train_set_sql_file).format(trx_end_date='2024-11-18',
                                                          time_period = 7,
                                                          user_base_sql=user_base_sql)

In [43]:
query_to_bq(query=um_query, client_project_id=BQ_PROJECT_ID,
            table_uri=f'{BQ_PROJECT_ID}.{BQ_BASE_DATASET}.combined_user_list_UM',
            mode='overwrite')

In [82]:
df = pdgbq.read_gbq(
        um_query,
    use_bqstorage_api=True)

Downloading: 100%|██████████| 4247800/4247800 [00:06<00:00, 667042.25rows/s]


In [67]:
gt_df = pdgbq.read_gbq(
        gtum_query,
    use_bqstorage_api=True)

Downloading: 100%|██████████| 638413/638413 [00:02<00:00, 280927.04rows/s]


In [83]:
selected_items = [f'{k}_{i}' for k, v in selected_items_sar.items() for i in v]
print(len(selected_items))

26


In [84]:
item_trx_count = df.groupby(['item'])['count_txn'].sum().reset_index()
item_trx_count = item_trx_count.sort_values(['count_txn'], ascending=False)
item_to_remove_um = list(item_trx_count[item_trx_count['count_txn'] <= sar_trx_count_thres]['item'])
item_to_remove_um = [i for i in item_to_remove_um if i not in selected_items]
df = df[~df['item'].isin(item_to_remove_um)]

In [85]:
df['label'] = df['count_txn']
df['itemId'] = pd.factorize(df['item'])[0]
df['customerId'] = pd.factorize(df['customer_id'])[0]

In [86]:
id2item_df = df[['itemId', 'item']].drop_duplicates().reset_index(drop=True)
id2user_df = df[['customerId', 'customer_id']].drop_duplicates().reset_index(drop=True)
item2cat_df = df[['item', 'category']].drop_duplicates().reset_index(drop=True)

In [87]:
df["label"] = df["label"].astype(np.float32)

In [88]:
model = SAR(
    **sar_model_params
)
with Timer() as train_time:
    model.fit(df)
print("SAR: Took {} seconds for training.".format(train_time.interval))
logging.info("SAR: Took {} seconds for training.".format(train_time.interval))
df['label'] = df['count_txn']

SAR: Took 6.627217907996965 seconds for training.


In [89]:
prediction_df = create_predict_df(
    id2user_df, id2item_df, df,
    selected_items_sar
)

Num items to recommend = 26


In [90]:
with Timer() as test_time:
    predictions_sar_test = model.predict(prediction_df)
print(predictions_sar_test.shape)

(46256210, 3)


In [91]:
prediction_df = prediction_df.merge(predictions_sar_test, left_on = ['customerId','itemId'], right_on = ['customerId','itemId'], how = 'left')
prediction_df.rename(columns = {'prediction':'pred'},  inplace = True)
print(prediction_df.shape)

(46256210, 4)


In [92]:
prediction_df = prediction_df.merge(id2item_df, on=['itemId'], how='inner') \
                             .merge(id2user_df, on=['customerId'], how='inner') \
                             .reset_index(drop=True)

In [93]:
prediction_df

,customerId,itemId,label,pred,item,customer_id
0,0,112,0,0.036897,TOPUP_TOPUP,804657126
1,0,120,0,0.007325,ONLINE1PD_BPJS,804657126
2,0,121,0,0.005190,ONLINE1PD_GAMES,804657126
3,0,122,0,0.039915,ONLINE1PD_PLN,804657126
4,0,124,0,0.030492,ONLINE1PD_ELECTRONIC_MONEY,804657126
...,...,...,...,...,...,...
46256205,1779084,2539,0,0.017121,TRANSFER_TO_PAY,636579023
46256206,1779084,2541,0,0.013263,TRANSFER_TO_BANK,636579023
46256207,1779084,2542,0,0.001668,ONLINE3PD_GAMES_3PD,636579023
46256208,1779084,2543,0,0.004717,ONLINE3PD_Tiktok_Shop,636579023


In [1]:
# prediction_df = predictions_sar_test.merge(id2item_df, on=['itemId'], how='inner') \
#                                     .merge(id2user_df, on=['customerId'], how='inner') \
#                                     .reset_index(drop=True)

# prediction_df = prediction_df.merge(gt_df, on=['customer_id', 'item'], how='left')

In [78]:
prediction_df['label'] = prediction_df['count_txn'].fillna(0)
prediction_df['rank'] = prediction_df.groupby('customer_id')['prediction'].rank(
    ascending=False, method='first').astype(int)
prediction_df.rename(columns={'prediction': 'pred'}, inplace=True)

In [79]:
n_customers = prediction_df['customer_id'].nunique()

for i in range(1, 4):
    print(f'\n Rank {i}')
    tmp = prediction_df[prediction_df['rank'] == i]
    tmp = tmp.groupby('item')['customer_id'] \
             .count() \
             .sort_values(ascending=False) \
             .reset_index() \
             .rename(columns={'customer_id': 'num_customers'})
    tmp['pct_customers'] = tmp['num_customers'] / n_customers
    display(tmp)


 Rank 1


,item,num_customers,pct_customers
0,TRANSFER_TO_BANK,1391332,7.997053e-01
1,ONLINE1PD_PULSA,141091,8.109582e-02
2,TRANSFER_TO_PAY,119359,6.860478e-02
3,ONLINE3PD_Google_Playstore_3PD,35513,2.041205e-02
4,ONLINE3PD_Tiktok_Shop,20032,1.151393e-02
5,ONLINE3PD_GAMES_3PD,17815,1.023965e-02
6,ONLINE1PD_PLN,5754,3.307265e-03
7,TOPUP_TOPUP,2582,1.484074e-03
8,TRANSFER_TO_OTHER_E_WALLET,2557,1.469704e-03
9,ONLINE1PD_ELECTRONIC_MONEY,1206,6.931807e-04



 Rank 2


,item,num_customers,pct_customers
0,TRANSFER_TO_PAY,1345436,0.773325
1,TRANSFER_TO_BANK,185624,0.106692
2,ONLINE1PD_PULSA,125531,0.072152
3,ONLINE3PD_GAMES_3PD,24561,0.014117
4,ONLINE3PD_Google_Playstore_3PD,23569,0.013547
5,ONLINE1PD_PLN,21131,0.012146
6,ONLINE3PD_Tiktok_Shop,7212,0.004145
7,TOPUP_TOPUP,2895,0.001664
8,TRANSFER_TO_OTHER_E_WALLET,1402,0.000806
9,ONLINE1PD_ELECTRONIC_MONEY,998,0.000574



 Rank 3


,item,num_customers,pct_customers
0,ONLINE1PD_PULSA,1446574,8.314571e-01
1,TRANSFER_TO_PAY,115139,6.617922e-02
2,TRANSFER_TO_BANK,113794,6.540614e-02
3,ONLINE3PD_Google_Playstore_3PD,23987,1.378717e-02
4,ONLINE1PD_PLN,17448,1.002870e-02
5,ONLINE3PD_GAMES_3PD,8110,4.661439e-03
6,ONLINE3PD_Tiktok_Shop,7428,4.269442e-03
7,TRANSFER_TO_OTHER_E_WALLET,2978,1.711685e-03
8,TOPUP_TOPUP,1458,8.380245e-04
9,ONLINE1PD_CABLE_TV_AND_INTERNET,936,5.379910e-04


In [80]:
 ndcg_1 = metric_calc_spark(prediction_df, ndcg_k=1)
print(f"NDCG@1: {ndcg_1}")
ndcg_3 = metric_calc_spark(prediction_df, ndcg_k=3)
print(f"NDCG@3: {ndcg_3}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/21 10:48:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


NDCG@1: 0.610421402496874


NDCG@3: 0.7391653648448573


In [94]:
prediction_df['rank'] = prediction_df.groupby('customer_id')['pred'].rank(
    ascending=False, method='first').astype(int)

In [95]:
prediction_df = prediction_df[['customer_id', 'item', 'pred', 'rank']]
prediction_df = prediction_df.rename(columns={
    'pred': 'score',
    'item': 'category'
})

In [96]:
prediction_df

,customer_id,category,score,rank
0,804657126,TOPUP_TOPUP,0.036897,7
1,804657126,ONLINE1PD_BPJS,0.007325,15
2,804657126,ONLINE1PD_GAMES,0.005190,18
3,804657126,ONLINE1PD_PLN,0.039915,6
4,804657126,ONLINE1PD_ELECTRONIC_MONEY,0.030492,9
...,...,...,...,...
46256205,636579023,TRANSFER_TO_PAY,0.017121,1
46256206,636579023,TRANSFER_TO_BANK,0.013263,2
46256207,636579023,ONLINE3PD_GAMES_3PD,0.001668,9
46256208,636579023,ONLINE3PD_Tiktok_Shop,0.004717,5


In [ ]:
preds = prediction_df['rank']

In [97]:
pd.DataFrame(prediction_df[prediction_df['rank']==1]['category'].value_counts())

,category
TRANSFER_TO_BANK,1411585
ONLINE1PD_PULSA,154236
TRANSFER_TO_PAY,125835
ONLINE3PD_Google_Playstore_3PD,29150
ONLINE3PD_Tiktok_Shop,20770
ONLINE3PD_GAMES_3PD,20715
ONLINE1PD_PLN,6276
TOPUP_TOPUP,3324
TRANSFER_TO_OTHER_E_WALLET,2548
ONLINE1PD_ELECTRONIC_MONEY,1359


In [98]:
BQ_BASE_DATASET , BQ_PROJECT_ID

('gopay_genie', 'gopay-ds-staging')

In [99]:
save_to_bigquery(dataset='playground_us',
                 data=prediction_df,
                 tablename='UM_prediction_result',
                 project='gopay-ds-staging')

In [109]:
DFS = pdgbq.read_gbq(
        f"""
        select a.* ,
            MAX(CASE WHEN b.rank = 1 THEN b.category else NULL end) AS rank_1_uc,
            MAX(CASE WHEN b.rank = 2 THEN b.category else NULL end) AS rank_2_uc,
            MAX(CASE WHEN b.rank = 3 THEN b.category else NULL end) AS rank_3_uc
        from `gopay-ds-staging.playground_us.combined_user_list` a
        join `gopay-ds-staging.playground_us.UM_prediction_result` b on a.customer_id = b.customer_id
        where b.rank < 4
        group by 1,2,3,4
        """,
    use_bqstorage_api=True)

Downloading: 100%|██████████| 1779085/1779085 [00:02<00:00, 662990.52rows/s] 


In [111]:
DFS.isna().sum()

customer_id       0
food_treatment    0
ride_treatment    0
car_treatment     0
rank_1_uc         0
rank_2_uc         0
rank_3_uc         0
dtype: int64

In [110]:
DFS = DFS.replace("",0)

,customer_id,food_treatment,ride_treatment,car_treatment,rank_1_uc,rank_2_uc,rank_3_uc
0,615537747,30p-30k-cashback,DD x 1 x 30% x 15000 x 0,1 x 30% x 8000,TRANSFER_TO_PAY,TRANSFER_TO_BANK,ONLINE1PD_PULSA
1,800825313,70p-20k-discount,DD x 1 x 30% x 15000 x 0,1 x 50% x 6000,ONLINE1PD_PULSA,TRANSFER_TO_BANK,TRANSFER_TO_PAY
2,796157027,70p-20k-discount,DD x 1 x flat x 10000 x 10000,0,TRANSFER_TO_BANK,TRANSFER_TO_PAY,ONLINE1PD_PULSA
3,549035909,25p-25k-discount,DD x 1 x 30% x 15000 x 0,1 x 30% x 10000,TRANSFER_TO_BANK,TRANSFER_TO_PAY,ONLINE1PD_PULSA
4,804765892,70p-20k-discount,DD x 1 x flat x 10000 x 10000,1 x 30% x 10000,TRANSFER_TO_BANK,TRANSFER_TO_PAY,ONLINE1PD_PULSA
...,...,...,...,...,...,...,...
1779080,823758286,70p-20k-discount,DD x 1 x flat x 10000 x 15000,0,TRANSFER_TO_BANK,TRANSFER_TO_PAY,ONLINE1PD_PULSA
1779081,825411812,70p-20k-discount,DD x 1 x flat x 10000 x 15000,0,TRANSFER_TO_BANK,TRANSFER_TO_PAY,ONLINE1PD_PULSA
1779082,608740818,70p-20k-discount,DD x 1 x flat x 10000 x 15000,1 x 30% x 8000,TRANSFER_TO_BANK,TRANSFER_TO_PAY,ONLINE1PD_PULSA
1779083,834545691,70p-20k-discount,DD x 1 x flat x 10000 x 15000,1 x 70% x 4000,ONLINE1PD_PULSA,TRANSFER_TO_BANK,TRANSFER_TO_PAY


In [103]:
DFS[DFS['rank']<4]

,customer_id,food_treatment,ride_treatment,car_treatment,customer_id_1,category,score,rank,rank_1_uc,rank_2_uc,rank_3_uc
7,829181928,70p-20k-discount,DD x 1 x 70% x 15000 x 15000,1 x 50% x 10000,829181928,ONLINE1PD_PULSA,0.193750,3,None,None,ONLINE1PD_PULSA
8,829181928,70p-20k-discount,DD x 1 x 70% x 15000 x 15000,1 x 50% x 10000,829181928,TRANSFER_TO_BANK,0.334789,1,TRANSFER_TO_BANK,None,None
20,623073599,70p-20k-discount,DD x 1 x 70% x 15000 x 15000,1 x 50% x 10000,623073599,TRANSFER_TO_PAY,0.149942,2,None,TRANSFER_TO_PAY,None
31,831233552,70p-20k-discount,DD x 1 x 70% x 20000 x 25000,1 x 50% x 10000,831233552,ONLINE1PD_PULSA,0.023139,3,None,None,ONLINE1PD_PULSA
37,813687543,25p-25k-discount,DD x 1 x 70% x 20000 x 25000,1 x 50% x 10000,813687543,TRANSFER_TO_BANK,0.040918,1,TRANSFER_TO_BANK,None,None
...,...,...,...,...,...,...,...,...,...,...,...
46256153,678372263,70p-20k-discount,DD x 1 x 70% x 10000 x 0,1 x 50% x 6000,678372263,TRANSFER_TO_PAY,0.844689,2,None,TRANSFER_TO_PAY,None
46256171,797594717,70p-20k-discount,DD x 1 x 70% x 10000 x 0,1 x 50% x 6000,797594717,TRANSFER_TO_PAY,0.140917,2,None,TRANSFER_TO_PAY,None
46256175,785058457,70p-20k-discount,DD x 1 x 70% x 10000 x 0,1 x 50% x 6000,785058457,TRANSFER_TO_BANK,0.179473,1,TRANSFER_TO_BANK,None,None
46256192,716369225,70p-20k-discount,DD x 1 x 70% x 10000 x 0,1 x 50% x 6000,716369225,ONLINE1PD_PULSA,0.107392,2,None,ONLINE1PD_PULSA,None


In [112]:
save_to_bigquery(dataset='playground_us',
                 data=DFS,
                 tablename='UM_prediction_result2',
                 project='gopay-ds-staging')

In [ ]:
queryy = f"""
        with all_table as(
                    select distinct customer_id,
                    order_no, country_code,
                    case 
                        when service_type_name LIKE '%GO-RIDE%' THEN "RIDE"
                        when service_type_name LIKE '%GO-CAR%' THEN "CAR"
                        when service_type_name LIKE '%GO_FOOD%' THEN "FOOD"
                    END AS service_type_name,
                    jakarta_data_date,
                    location.booking_destination_location_name AS DESTINATION_NAME,
                    booking_pickup_location_name AS PICKUP_NAME
                    FROM `data-gojek-id-mart.data_warehouse.detail_all_booking`, UNNEST(booking_destination_location_list) AS location
                    where jakarta_data_date >= "2024-11-14"
                    AND jakarta_data_date <= "2024-11-20"
                    AND lower(business_status_name)='completed'
                    AND country_code = 'ID'
                    and customer_id is not null
                    and (service_type_name LIKE '%GO-RIDE%' or service_type_name LIKE '%GO-CAR%' or service_type_name LIKE '%GO_FOOD%')
        )
        
        , cte_1 AS (
                select distinct customer_id, 
                    service_type_name,
                    DESTINATION_NAME AS rec_place,
                    COUNT(*) as total_trip,
                   
                FROM all_table
                WHERE service_type_name IN ('RIDE', 'CAR')
                GROUP BY 1,2,3
                
                UNION ALL
                
                select distinct customer_id, 
                    service_type_name,
                    PICKUP_NAME AS rec_place,
                    COUNT(*) as total_trip,
                FROM all_table
                WHERE service_type_name IN ('FOOD')
                GROUP BY 1,2,3
            )
            
        , cte2 as(
        select *, RANK() OVER (PARTITION BY customer_id ORDER BY total_trip DESC) AS rank from cte_1
        )
        
       ,cte3 as(
            select * from cte2 
            where rank = 1
        )
        
            select a.*,
            case when b.service_type_name = 'RIDE' THEN b.rec_place ELSE NULL END AS RIDE_PREDICTION_PLACE,
            case when b.service_type_name = 'CAR' THEN b.rec_place ELSE NULL END AS CAR_PREDICTION_PLACE,
            case when b.service_type_name = 'FOOD' THEN b.rec_place ELSE NULL END AS FOOD_PREDICTION_PLACE
            from `gopay-ds-staging.playground_us.UM_prediction_result2` a
            left join cte3 b  on a.customer_id = b.customer_id
        
        
        
       """

In [ ]:
query_to_bq(query=queryy, client_project_id="gopay-ds-staging",
            table_uri=f'gopay-ds-staging.playground_us.UM_prediction_result3',
            mode='overwrite')

In [232]:
transacting_gojek_app_users[(~transacting_gojek_app_users['RIDE_PREDICTION_PLACE'].isnull()) | (~transacting_gojek_app_users['CAR_PREDICTION_PLACE'].isnull())]

,customer_id,food_treatment,ride_treatment,car_treatment,rank_1_uc,rank_2_uc,rank_3_uc,RIDE_PREDICTION_PLACE,CAR_PREDICTION_PLACE
10,567410489,70p-20k-discount,0,0,TRANSFER_TO_BANK,TOPUP_TOPUP,ONLINE3PD_Google_Playstore_3PD,None,Jl. Jaring 8 No.179
47,812622493,70p-20k-discount,DD x 1 x 70% x 15000 x 0,0,TRANSFER_TO_PAY,ONLINE3PD_Tiktok_Shop,TRANSFER_TO_BANK,Terminal Penumpang Nusantara,None
49,706529944,70p-20k-discount,DD x 1 x 70% x 15000 x 0,0,TRANSFER_TO_BANK,TRANSFER_TO_OTHER_E_WALLET,TRANSFER_TO_PAY,Kreasi Prima Land,None
50,706529944,70p-20k-discount,DD x 1 x 70% x 15000 x 0,0,TRANSFER_TO_BANK,TRANSFER_TO_OTHER_E_WALLET,TRANSFER_TO_PAY,Stasiun Tambun,None
53,825482693,70p-20k-discount,DD x 1 x 70% x 15000 x 0,0,TRANSFER_TO_PAY,TRANSFER_TO_OTHER_E_WALLET,TRANSFER_TO_BANK,RT 01,None
...,...,...,...,...,...,...,...,...,...
1929177,576713832,,0,1 x 70% x 4000,TRANSFER_TO_BANK,ONLINE3PD_Google_Playstore_3PD,TRANSFER_TO_PAY,None,Bandara Internasional Minangkabau
1929178,576713832,,0,1 x 70% x 4000,TRANSFER_TO_BANK,ONLINE3PD_Google_Playstore_3PD,TRANSFER_TO_PAY,SMA Katolik Xaverius Padang,None
1929184,609720106,25p-25k-discount,0,1 x 70% x 4000,TRANSFER_TO_BANK,ONLINE3PD_Google_Playstore_3PD,TRANSFER_TO_PAY,Kios unhy,None
1929220,612476014,30p-30k-cashback,DD x 1 x 50% x 20000 x 25000,1 x 30% x 10000,TRANSFER_TO_BANK,ONLINE3PD_Google_Playstore_3PD,TRANSFER_TO_PAY,None,The Jayakarta Suite Hotel & Resort
